# Scraping data from product webpages into text files

In [2]:
#import libaries
from urllib.request import urlopen, Request

In [5]:
#import libaries
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import numpy as np
import pandas as pd

In [3]:
#import libaries
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re

In [6]:
#used selenium and chrome to automate a web browser to scroll through webpage since it couldn't be scraped by normal means

#append links to each product on the website to list 'Wow'
wow = []

driver = webdriver.Chrome('C:\\Users\\akomandu\\Python_Stuff\\chromedriver.exe')
driver.get("https://living-future.org/declare/")
driver.maximize_window()

SCROLL_PAUSE_TIME = 1.0

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    wow.append(soup.find_all('a', href=True))
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [7]:
Links = []

#filter out extraneous links
for i in wow[-1]:
    if 'lf-reveal-card' in str(i):
        Links.append(i['href'])

In [ ]:
#for every link, gather product information and write it to text file
for link in Links:
    print(link)
    page = requests.get(link)
    time.sleep(5)
    soup = BeautifulSoup(page.content, 'html.parser')

    company = soup.find('strong').get_text()
    extra = []
    for i in soup.findAll("div", {"class": "declare-option"}):
        extra.append(i.get_text().split('\n'))
    
    with urlopen(Request(link, headers={'User-Agent': 'Mozilla'})) as url:
        s=url.read()
        dfs = pd.read_html(s)
    
    f = open(link.split('/')[-2] + '.txt', 'w', encoding='utf-8')
    j=0
    f.write(company)
    for i in extra:
        i[1] = i[1] + ':'
        f.write(' '.join(i) + '\n')
    f.write('\n')
    f.write(' '.join(list(dfs[0])) + '\n')
    for index,row in dfs[0].iterrows():
        if j > len(dfs):
            break
        else:
            f.write(str(row[0]) + '__' + str(row[1]) + '__' + str(row[2]) + '__' + str(row[3]) + '__' + str(row[4]))
            f.close()
            j+=1


# Converting Text Files to PDF Files

In [1]:
import os     ## Allows operating system commands to be performed though Python
import sys    ## Allows you to exit a Python script for testing
import shutil ## Allows movement of files from one directory to another
import  glob  ## Global searching of strings as files

In [2]:
os.chdir('C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Living_Future\\Files')

In [6]:
#gather text files
files = glob.glob('C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Living_Future\\Files\\*.txt')

In [33]:
#import libraries for conversion
import reportlab
from textwrap import wrap
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen.canvas import Canvas

In [68]:
#switch to new folder to put PDFs in
os.chdir('C:\\Users\\akomandu\\Python_Portfolio\\PDF_Scrapes\\Living_Future\\PDFs')

In [ ]:
#create PDFs
for i in files:
    f = open(i, encoding='utf-8')
    title = str(i.split('\\')[-1][:-3] + 'pdf')
    text = f.read()
    okay = text.split('\n')
    for n,i in enumerate(okay):
        if 'Description:' in i:
            okay[n] = '\n'.join(wrap(i, 100))
    wrapped = '\n'.join(okay)
    print(title)
    c = Canvas(title, pagesize=A4)
    t = c.beginText()
    t.setTextOrigin(50, 800)
    t.textLines(wrapped)
    c.drawText(t)
    c.save()